# 6.2 데이터 전처리

In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.DataFrame([
    [42, 'male', 12,'reading','class2'],
    [35, 'unknown', 3,'cooking', 'class1'],
    [1000, 'female', 7,'cycling', 'class3'],
    [1000, 'unknown', 21,'unknown', 'unknown']
])
df.columns=['age',
            'gender',
            'month_birth',
            'hobby','target']

In [3]:
df

,age,gender,month_birth,hobby,target
0,42,male,12,reading,class2
1,35,unknown,3,cooking,class1
2,1000,female,7,cycling,class3
3,1000,unknown,21,unknown,unknown


### 결측치 변경에 앞서 각 열의 유니크 값 확인

In [4]:
df['age'].unique()

array([  42,   35, 1000], dtype=int64)

In [5]:
df['gender'].unique()

array(['male', 'unknown', 'female'], dtype=object)

In [6]:
df['month_birth'].unique()

array([12,  3,  7, 21], dtype=int64)

In [7]:
df['hobby'].unique()

array(['reading', 'cooking', 'cycling', 'unknown'], dtype=object)

In [8]:
df['target'].unique()

array(['class2', 'class1', 'class3', 'unknown'], dtype=object)

### 결측치 처리

In [9]:
df.loc[df['age']>150, ['age']] = np.nan
df.loc[df['gender']=='unknown', ['gender']] = np.nan
df.loc[df['month_birth']>12, ['month_birth']] = np.nan
df.loc[df['hobby']=='unknown', ['hobby']] = np.nan
df.loc[df['target']=='unknown', ['target']] = np.nan
df

,age,gender,month_birth,hobby,target
0,42.0,male,12.0,reading,class2
1,35.0,NaN,3.0,cooking,class1
2,NaN,female,7.0,cycling,class3
3,NaN,NaN,NaN,NaN,NaN


데이터 셋을 처음 접했을 때는 각 열에 결측치가 존재하는지 확인.
부적절한 값도 결측치로 처리하는 것이 좋음.

In [10]:
# 데이터 셋의 각 열의 결측치 개수
df.isnull().sum()

age            2
gender         2
month_birth    1
hobby          1
target         1
dtype: int64

In [11]:
# 결측치를 포함한 행 삭체
df2 = df.dropna(axis=0)
df2

,age,gender,month_birth,hobby,target
0,42.0,male,12.0,reading,class2


In [12]:
# 결측치를 포함한 열 삭제
df3 = df.dropna(axis=1)
df3

""
0
1
2
3


In [13]:
# 모든 값이 결측치인 행 삭제
df4 = df.dropna(how='all')
df4

,age,gender,month_birth,hobby,target
0,42.0,male,12.0,reading,class2
1,35.0,NaN,3.0,cooking,class1
2,NaN,female,7.0,cycling,class3


In [14]:
# 값이 2개 미만인 행 삭제
df5 = df.dropna(thresh=2)
df5

,age,gender,month_birth,hobby,target
0,42.0,male,12.0,reading,class2
1,35.0,NaN,3.0,cooking,class1
2,NaN,female,7.0,cycling,class3


In [15]:
# 특정 열에 결측치가 있는 경우 행 삭제
df6 = df.dropna(subset=['gender'])
df6

,age,gender,month_birth,hobby,target
0,42.0,male,12.0,reading,class2
2,NaN,female,7.0,cycling,class3


In [16]:
# 결측치 대체하기
alter_values = {'age': 0, 
                'gender': 'U', 
                'month_birth': 0,
                'hobby': 'U',
                'target': 'class4'}
df7 = df.fillna(value = alter_values)
df7

,age,gender,month_birth,hobby,target
0,42.0,male,12.0,reading,class2
1,35.0,U,3.0,cooking,class1
2,0.0,female,7.0,cycling,class3
3,0.0,U,0.0,U,class4


### 클래스 라벨 설정
해당 피처 데이터가 나타내는 게 무엇인지 이름표를 붙이는 것
target 열이 문자열(string)인 경우 정수(int)로 바꿔준다.

In [17]:
from sklearn.preprocessing import LabelEncoder
df8 = df7
# LabelEncoder()를 이용해 라벨링 모형을 설정
class_label = LabelEncoder()
# 타겟 변수의 값 불러옴
data_value = df8['target'].values
data_value

array(['class2', 'class1', 'class3', 'class4'], dtype=object)

In [18]:
# 라벨링 모형에 데이터값을 넣고 변환
y_new = class_label.fit_transform(data_value)
y_new

array([1, 0, 2, 3])

In [19]:
# 클래스 라벨링된 결과 확인
df8['target'] = y_new
df8

,age,gender,month_birth,hobby,target
0,42.0,male,12.0,reading,1
1,35.0,U,3.0,cooking,0
2,0.0,female,7.0,cycling,2
3,0.0,U,0.0,U,3


In [20]:
# 클래스 라벨링 복구
y_ori = class_label.inverse_transform(y_new)
df8['target'] = y_ori
df8

,age,gender,month_birth,hobby,target
0,42.0,male,12.0,reading,class2
1,35.0,U,3.0,cooking,class1
2,0.0,female,7.0,cycling,class3
3,0.0,U,0.0,U,class4


In [21]:
# 직접 클래스 라벨링 하는 방법
y_arr = df8['target'].values
y_arr.sort()
y_arr

array(['class1', 'class2', 'class3', 'class4'], dtype=object)

In [22]:
num_y = 0
dic_y = {}
for ith_y in y_arr:
    dic_y[ith_y] = num_y
    num_y += 1
dic_y # 딕셔너리를 이용해 각 데이터값을 변경 값으로 맵핑

{'class1': 0, 'class2': 1, 'class3': 2, 'class4': 3}

In [23]:
df8['target'] = df8['target'].replace(dic_y)
df8

,age,gender,month_birth,hobby,target
0,42.0,male,12.0,reading,0
1,35.0,U,3.0,cooking,1
2,0.0,female,7.0,cycling,2
3,0.0,U,0.0,U,3


### 원-핫 인코딩(one-hot encoding)
오직 0과 1만 사용한 벡터를 이용해 데이터값을 나타내는 방법

In [24]:
df9 = df8
df9['target'] = df9['target'].astype(str)
df10 = pd.get_dummies(df9['target'])
print(df10)

   0  1  2  3
0  1  0  0  0
1  0  1  0  0
2  0  0  1  0
3  0  0  0  1


In [25]:
# drop_first=True로 지정하면 맨 처음 범주에 대해서는 별도로 dummy variable을 할당하지 않음
df9['target'] = df9['target'].astype(str)
df11 = pd.get_dummies(df9['target'], drop_first=True) 
print(df11)

   1  2  3
0  0  0  0
1  1  0  0
2  0  1  0
3  0  0  1


In [26]:
df12 = df8
df13 = pd.get_dummies(df12)
df13

,age,month_birth,gender_U,gender_female,gender_male,hobby_U,hobby_cooking,hobby_cycling,hobby_reading,target_0,target_1,target_2,target_3
0,42.0,12.0,0,0,1,0,0,0,1,1,0,0,0
1,35.0,3.0,1,0,0,0,1,0,0,0,1,0,0
2,0.0,7.0,0,1,0,0,0,1,0,0,0,1,0
3,0.0,0.0,1,0,0,1,0,0,0,0,0,0,1


(1) 사이킷런 라이브러리를 이용한 원-핫 인코딩

In [27]:
from sklearn.preprocessing import OneHotEncoder
hot_encoder = OneHotEncoder()
y = df7[['target']]
y_hot = hot_encoder.fit_transform(y)
y_hot.toarray()

array([[1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.]])

(2) 텐서플로 라이브러리를 이용한 원-핫 인코딩

In [28]:
from tensorflow.keras.utils import to_categorical
y_hotec = to_categorical(y) 
y_hotec

array([[1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.]], dtype=float32)

### 데이터 스케일링
데이터 값이 단위에 영향을 받지 않도록 변형하는 것
스케일링 기준값: 평균, 분산, 최댓값, 최솟값

(1) 표준화 스케일링: 데이터가 평균 0, 표준 편차 1이 되도록 변경하는 방법

In [29]:
# 표준화 스케일링
from sklearn.preprocessing import StandardScaler

# 다 풀어 쓰면
std = StandardScaler()
std.fit(df8[['month_birth']])
x_std = std.transform(df8[['month_birth']])

# 축약하면
x_std2 = std.fit_transform(df8[['month_birth']])

In [30]:
print("mean: %.6f, standard deviation: %.6f"%(np.mean(x_std), np.std(x_std)))

mean: -0.000000, standard deviation: 1.000000


(2) 로버스트 스케일링: 중앙값과 사분위수를 사용, 극단값의 영향을 거의 받지 않는다는 장점이 있음

In [31]:
# 로버스트 스케일링
from sklearn.preprocessing import RobustScaler

robust= RobustScaler()
robust.fit(df8[['month_birth']])
x_robust = robust.transform(df8[['month_birth']])
x_robust

array([[ 1.16666667],
       [-0.33333333],
       [ 0.33333333],
       [-0.83333333]])

(3) 최소-최대 스케일링

In [32]:
# 최소-최대 스케일링
from sklearn.preprocessing import MinMaxScaler

minmax = MinMaxScaler()
minmax.fit(df8[['month_birth']])
x_minmax = minmax.transform(df8[['month_birth']])
x_minmax

array([[1.        ],
       [0.25      ],
       [0.58333333],
       [0.        ]])

(4) 노멀 스케일링: 벡터의 유클리디안 길이가 1이 되도록 데이터값을 변경
앞의 것과 달리 노멀 스케일은 행 기준
노멀 스케일링은 normalization이라고도 함

In [33]:
# 노멀 스케일링
from sklearn.preprocessing import Normalizer

normal = Normalizer()
normal.fit(df8[['age','month_birth']])
x_normal = normal.transform(df8[['age','month_birth']])
x_normal

array([[0.96152395, 0.27472113],
       [0.99634665, 0.08540114],
       [0.        , 1.        ],
       [0.        , 0.        ]])